In [1]:
# auto-refresh helper functions
%load_ext autoreload
%autoreload 2

In [48]:
from scipy.spatial import KDTree
import pynndescent
import numpy as np
import cvxpy as cp
from sklearn.base import BaseEstimator, ClassifierMixin

class DKNN(BaseEstimator, ClassifierMixin):
    def __init__(self, k, alpha=1, beta=1):
        super().__init__()
        self.k = k          # 'k' neighbors
        self.A = None       # PSD matrix objective
        self.pi = None      # technically log(pi)
        self.trees = []     # search tree for NN
        
        # importance weights for each class (k,)
        if type(alpha) in {float, int}:
            self.alpha = np.full(k, alpha)
        else:
            assert k == len(alpha)
            self.alpha = alpha
        self.beta  = beta   # regularization term

    # Mahalanobis distance
    def dist(self, x, mu, c):
        delta = x - mu
        return np.sum(np.multiply(delta @ self.A, delta), axis=-1) - self.pi[c]

    def fit(self, X, y):
        self.trees = [] # reset for each fit -- when using CV need this
        self.X = X
        self.C = np.unique(y)
        self.classes_ = self.C
        self.c_idx = []  # indices belonging to 'c' w.r.t full training X
        for ci in self.C:
            self.c_idx.append(np.where(y == ci))
        n, d = X.shape

        centroids = []

        # Find centroids of class C[i]
        for idx in self.c_idx:
            # Get k nearest neighbors of class C[i] for all training data X

            # tree = KDTree(X[idx])
            # _, n_idx = tree.query(X, self.k)
            # self.trees.append(tree)

            index = pynndescent.NNDescent(X[idx])
            index.prepare()
            self.trees.append(index)

            n_idx, _ = index.query(X)
            # Take k nearest neighbors
            n_idx = n_idx[:, :self.k]

            # Compute centroids
            neighbors = X[idx][n_idx] # X[of class 'c'][its nearest neighbors w.r.t X[c]]
            if self.k == 1:
                centroid_c = neighbors
            else:
                centroid_c = np.mean(neighbors, axis=1)
            centroids.append(centroid_c)
        
        centroids = np.stack(centroids, axis=0)

        # Convex problem formulation
        self.pi = np.array([len(idx[0]) / n for idx in self.c_idx])
        self.A = cp.Variable((d, d))

        delta = X - centroids

        # should work
        # f_mult = np.sum(np.multiply(delta @ self.A, delta), axis=2) - self.pi[:, np.newaxis]
        # print(f_mult[0, 0])

        constraints = []
        epsilon = cp.Variable(n)
        constraints.append(epsilon >= 0)

        for i in range(n):
            for c in self.C:
                if c == y[i]:
                    continue
                constraints += [
                    delta[y[i], i] @ self.A @ delta[y[i], i].T - cp.log(self.pi[y[i]]) + 1 - epsilon[i] 
                    <= delta[c, i] @ self.A @ delta[c   , i].T - cp.log(self.pi[c])
                ]
        
        # alpha_vec = np.array([self.alpha[i] for i in range(len(y))])  # corresponding class importance weight
        objective = cp.Minimize(cp.sum(cp.multiply(1, epsilon)) + self.beta * cp.norm(self.A, 1))

        prob = cp.Problem(objective, constraints)
        prob.solve()

        self.A = self.A.value

    def predict(self, X_new):
        if X_new.ndim == 1:
            n = 1
        else:
            n = X_new.shape[0]

        dist_c = np.empty((n, len(self.trees)))
        for c, t in enumerate(self.trees):
            # each tree 't' is already a subset of X conditioned on y=c
            # _, n_idx = t.query(X_new, self.k)
            n_idx, _ = t.query(X_new)
            # Take k nearest neighbors
            n_idx = n_idx[:, :self.k]


            # Compute centroids
            neighbors = self.X[self.c_idx[c][0][n_idx]]
            centroid = np.mean(neighbors, axis=-2)
            cur_dist = self.dist(X_new, centroid, c)
            dist_c[:, c] = cur_dist
        
        predictions = np.argmin(dist_c, axis=1)
        return predictions
    
    def score(self, X_test, y_test):
        y_pred = self.predict(X_test)
        return np.average(y_pred == y_test)
    
    def get_params(self, deep=False):
        return {
            'k': self.k,
            'alpha': self.alpha,
            'beta': self.beta,
        }
    
    def set_params(self, **params):
        for key, value in params.items():
            setattr(self, key, value)
        return self

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, PowerTransformer, MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.neighbors import NearestCentroid
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

class DataTransformer:
    def __init__(self):
        # self.pt = PowerTransformer(method="box-cox")
        # self.pt = PowerTransformer(method="yeo-johnson")
        self.pt = StandardScaler()
        # self.pt = MinMaxScaler()
        self.lda = LDA()

    def fit(self, X, y):
        X = self.pt.fit_transform(X)
        # self.lda.fit(X, y)

        return self

    def transform(self, X):
        X = self.pt.transform(X)
        # X = self.lda.transform(X)

        return X

In [ ]:
import numpy as np
import dataset_helpers as ds
from sklearn.model_selection import StratifiedShuffleSplit

X, y = ds.get_UCI_dataset(None, dataset_id=763)   # mines
# X, y = ds.get_UCI_dataset(None, dataset_id=697)     # student
y = np.ravel(y)

rng = np.random.default_rng(42)
subset_idx = rng.choice(X.shape[0], min(X.shape[0], 500), replace=False)
X = X[subset_idx]
y = y[subset_idx]

print(X.shape, y.shape, flush=True)

knn_acc = []
k = 3
beta_vals = np.arange(6)
sss = StratifiedShuffleSplit(n_splits=10, test_size=0.1, random_state=42)
for i, (train_index, test_index) in enumerate(sss.split(X, y)):
        tf = DataTransformer().fit(X[train_index], y[train_index])

        clf = KNN(k)
        clf.fit(tf.transform(X[train_index]), y[train_index])
        knn_acc.append(clf.score(tf.transform(X[test_index]), y[test_index]))
print(f"k = {k}, knn acc = {np.mean(knn_acc)}")
for beta in beta_vals:
    dknn_acc = []
    for i, (train_index, test_index) in enumerate(sss.split(X, y)):
        tf = DataTransformer().fit(X[train_index], y[train_index])

        clf = DKNN(k, beta=beta)
        clf.fit(tf.transform(X[train_index]), y[train_index])
        dknn_acc.append(clf.score(tf.transform(X[test_index]), y[test_index]))
    print(f"k = {k}, beta = {beta}, dknn acc = {np.mean(dknn_acc)}")

(338, 3) (338,)
k = 3, knn acc = 0.5058823529411764


In [ ]:
sss = StratifiedShuffleSplit(n_splits=10, test_size=0.3)
print("DKNN")
for k in range(3, 7):
    dknn_acc = []
    for i, (train_index, test_index) in enumerate(sss.split(X, y)):
        clf = DKNN(k)
        tf = DataTransformer().fit(X[train_index], y[train_index])

        clf.fit(tf.transform(X[train_index]), y[train_index])
        dknn_acc.append(clf.score(tf.transform(X[test_index]), y[test_index]))
        # clf.fit(X[train_index], y[train_index])
        # acc.append(clf.score(X[test_index], y[test_index]))
    print(f"k = {k}, acc = {np.mean(dknn_acc)}")
print("-----------------")
print("KNN")
for k in range(3, 7):
    dknn_acc = []
    for i, (train_index, test_index) in enumerate(sss.split(X, y)):
        clf = KNN(k)
        tf = DataTransformer().fit(X[train_index], y[train_index])

        clf.fit(tf.transform(X[train_index]), y[train_index])
        dknn_acc.append(clf.score(tf.transform(X[test_index]), y[test_index]))
    print(f"k = {k}, acc = {np.mean(dknn_acc)}")

print("-----------------")
print("Centroid")
dknn_acc = []
for i, (train_index, test_index) in enumerate(sss.split(X, y)):
    clf = NearestCentroid()
    tf = DataTransformer().fit(X[train_index], y[train_index])

    clf.fit(tf.transform(X[train_index]), y[train_index])
    dknn_acc.append(clf.score(tf.transform(X[test_index]), y[test_index]))
print(f"acc = {np.mean(dknn_acc)}")

DKNN
k = 3, acc = 0.9444444444444444
k = 4, acc = 0.9703703703703704
k = 5, acc = 0.961111111111111
k = 6, acc = 0.9629629629629628
-----------------
KNN
k = 3, acc = 0.9574074074074073
k = 4, acc = 0.9333333333333333
k = 5, acc = 0.9351851851851853
k = 6, acc = 0.951851851851852
-----------------
Centroid
acc = 0.9611111111111112


In [ ]:
import dataset_helpers as ds
from sklearn.model_selection import LeaveOneOut

def print_deltas(results, ds_name):
    def red_string(txt):
        return f"\033[91m{txt:3f}\033[0m"
    # prints deltas and marks red if delta is far from a single standard deviation
    d_svm = results['SVM'][0] - ds.orig_stats[ds_name]['SVM'][0] # [0] is mean [1] is std
    d_lda = results['LDA'][0] - ds.orig_stats[ds_name]['LDA'][0]
    d_knn = results['KNN'][0] - ds.orig_stats[ds_name]['KNN'][0]

    # color red if the deltas are too large
    d_svm = f"{d_svm:3f}" if abs(d_svm) <= ds.orig_stats[ds_name]['SVM'][1] else red_string(d_svm)
    d_lda = f"{d_lda:3f}" if abs(d_lda) <= ds.orig_stats[ds_name]['LDA'][1] else red_string(d_lda)
    d_knn = f"{d_knn:3f}" if abs(d_knn) <= ds.orig_stats[ds_name]['KNN'][1] else red_string(d_knn)

    print("paper deltas:\n"
          f"\tSVM: {d_svm}\n"
          f"\tLDA: {d_lda}\n"
          f"\tKNN: {d_knn}\n"
    )

# for all datasets we have, get baseline stats
for ds_name in ds.name_to_id.keys():
    paper_stats = ds.orig_stats[ds_name]
    print(f"Results for {ds_name}:\n")
    data, labels = ds.get_UCI_dataset(ds_name)
    result = ds.run_baseline_models(
        data, labels.ravel(),
        data_preprocessor=None,
        verbose=True)
    print_deltas(result, ds_name)




Results for ionosphere:

SVM Mean Accuracy (from 10 runs): 0.869 (std: 0.029)
LDA Mean Accuracy (from 10 runs): 0.862 (std: 0.029)
CART Mean Accuracy (from 10 runs): 0.877 (std: 0.020)
KNN Mean Accuracy (from 10 runs): 0.843 (std: 0.020)
paper deltas:
	SVM: 0.017868
	LDA: -0.002736
	KNN: -0.014604

Results for sonar:

SVM Mean Accuracy (from 10 runs): 0.775 (std: 0.048)
LDA Mean Accuracy (from 10 runs): 0.737 (std: 0.073)
CART Mean Accuracy (from 10 runs): 0.678 (std: 0.033)
KNN Mean Accuracy (from 10 runs): 0.798 (std: 0.037)
paper deltas:
	SVM: 0.023603
	LDA: -0.000492
	KNN: -0.066587

Results for balance scale:

SVM Mean Accuracy (from 10 runs): 0.915 (std: 0.022)
LDA Mean Accuracy (from 10 runs): 0.872 (std: 0.025)
CART Mean Accuracy (from 10 runs): 0.778 (std: 0.015)
KNN Mean Accuracy (from 10 runs): 0.796 (std: 0.019)
paper deltas:
	SVM: 0.015894
	LDA: 0.163340
	KNN: -0.078723

Results for wine:

SVM Mean Accuracy (from 10 runs): 0.931 (std: 0.019)
LDA Mean Accuracy (from 10 runs

In [ ]:

# NOTE: everything below evaluated with LOOCV rather than the 70/30 split like above
# (following the paper)

dknn_clf = DKNN(k=6)
for dim in range(2, 5): # 2 to 4D synthetic datasets
    data, labels = ds.get_synthetic_dataset(dim, seed=None)
    loo = LeaveOneOut()
    dknn_acc = []
    for i, (train_index, test_index) in enumerate(loo.split(data, labels)):
        dknn_clf.fit(data[train_index], labels[train_index].astype(int))
        dknn_acc.append(dknn_clf.score(data[test_index], labels[test_index]))
    print(f"\n{dim}-dim DKNN(mean: {np.mean(dknn_acc):3f}, std: {np.std(dknn_acc):3f})\n")
    # baselines
    baseline_results = ds.run_baseline_models(
        data, labels.ravel(),
        data_preprocessor=None,
        verbose=True,
        loocv=True)
    # print(f"SVM : {baseline_results['SVM'][0]:3f}")
    # print(f"LDA : {baseline_results['LDA'][0]:3f}")
    # print(f"CART: {baseline_results['CART'][0]:3f}")
    # print(f"KNN : {baseline_results['KNN'][0]:3f}")

    



2-dim DKNN(mean: 0.806250, std: 0.395235)

SVM Mean Accuracy (from 10 runs): 0.850 (std: 0.357)
LDA Mean Accuracy (from 10 runs): 0.838 (std: 0.369)
CART Mean Accuracy (from 10 runs): 0.731 (std: 0.443)
KNN Mean Accuracy (from 10 runs): 0.794 (std: 0.405)

3-dim DKNN(mean: 0.837500, std: 0.368909)

SVM Mean Accuracy (from 10 runs): 0.869 (std: 0.338)
LDA Mean Accuracy (from 10 runs): 0.875 (std: 0.331)
CART Mean Accuracy (from 10 runs): 0.769 (std: 0.422)
KNN Mean Accuracy (from 10 runs): 0.831 (std: 0.375)

4-dim DKNN(mean: 0.868750, std: 0.337674)

SVM Mean Accuracy (from 10 runs): 0.887 (std: 0.316)
LDA Mean Accuracy (from 10 runs): 0.887 (std: 0.316)
CART Mean Accuracy (from 10 runs): 0.881 (std: 0.323)
KNN Mean Accuracy (from 10 runs): 0.831 (std: 0.375)
